In [1]:
from gensim import corpora, models, similarities, matutils
import numpy as np
import os 

In [2]:
# documents = ["Human machine interface for lab abc computer applications",
#              "A survey of user opinion of computer system response time",
#              "The EPS user interface management system",
#              "System and human system engineering testing of EPS",
#              "Relation of user perceived response time to error measurement",
#              "The generation of random binary unordered trees",
#              "The intersection graph of paths in trees",
#              "Graph minors IV Widths of trees and well quasi ordering",
#               "Graph minors A survey"]

In [3]:
if (os.path.exists("tmp/deerwester.dict")):
    dictionary = corpora.Dictionary.load('tmp/deerwester.dict')
    corpus = corpora.MmCorpus('tmp/deerwester.mm')
    print("Used files generated from first tutorial")
else:
    print("Please run first tutorial to generate data set")



Used files generated from first tutorial


In [4]:
for d in corpus:
    print(d)

[(0, 1.0), (1, 1.0), (2, 1.0)]
[(0, 1.0), (3, 1.0), (4, 1.0), (5, 1.0), (6, 1.0), (7, 1.0)]
[(2, 1.0), (5, 1.0), (7, 1.0), (8, 1.0)]
[(1, 1.0), (5, 2.0), (8, 1.0)]
[(3, 1.0), (6, 1.0), (7, 1.0)]
[(9, 1.0)]
[(9, 1.0), (10, 1.0)]
[(9, 1.0), (10, 1.0), (11, 1.0)]
[(4, 1.0), (10, 1.0), (11, 1.0)]


In [5]:
scipy_csc_matrix = matutils.corpus2csc(corpus)
print(scipy_csc_matrix.todense())

[[1. 1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 1. 0. 0. 0. 0. 0.]
 [1. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 1.]
 [0. 1. 1. 2. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 1. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 1. 1. 0.]
 [0. 0. 0. 0. 0. 0. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 1. 1.]]


In [6]:
print(dictionary.token2id)

{'computer': 0, 'human': 1, 'interface': 2, 'response': 3, 'survey': 4, 'system': 5, 'time': 6, 'user': 7, 'eps': 8, 'trees': 9, 'graph': 10, 'minors': 11}


# Budujemy model LDA
Budujemy model LDA i transformujemy dane

* **num_topics=2** oznacza ilość modelowanych tematów

In [7]:
model = models.LdaModel(corpus, id2word=dictionary, num_topics=2)
corpus_lda = model[corpus] # create a double wrapper over the original corpus: bow->tfidf->fold-in-lsi
for d in corpus_lda:
    print(d)

[(0, 0.83729), (1, 0.16271001)]
[(0, 0.90050095), (1, 0.09949899)]
[(0, 0.19252324), (1, 0.80747676)]
[(0, 0.12373279), (1, 0.8762672)]
[(0, 0.8622306), (1, 0.13776943)]
[(0, 0.3121528), (1, 0.6878472)]
[(0, 0.19222331), (1, 0.8077767)]
[(0, 0.14043865), (1, 0.8595613)]
[(0, 0.15467012), (1, 0.8453299)]


### Dla każdego dokumentu dostajemy prawdopodobieństwo przynależności dokumentu do danego tematu.

Możemy też zobaczyć z czego składają się tematy:

In [8]:
model.show_topics()

[(0,
  '0.124*"user" + 0.122*"computer" + 0.120*"response" + 0.115*"time" + 0.082*"system" + 0.082*"interface" + 0.078*"survey" + 0.077*"human" + 0.076*"trees" + 0.050*"graph"'),
 (1,
  '0.157*"system" + 0.139*"graph" + 0.116*"trees" + 0.108*"eps" + 0.103*"minors" + 0.074*"user" + 0.070*"human" + 0.069*"survey" + 0.066*"interface" + 0.037*"time"')]

# Zad

Chcemy postortowac słowa każdego tematu i wybrać 5 najważniejszych - co można powiedzieć o tematach?

Proszę zobaczyć na funkcje typu get_topics(), get_term_topics(...): https://radimrehurek.com/gensim/models/ldamodel.html

In [9]:
import numpy as np
topics = np.argsort(model.get_topics()[0,:])[::-1] #::-1 sortowanie w odwrotnej kolejności
for x in topics[:5]:
    print(dictionary[x])

user
computer
response
time
system


In [10]:
model.get_topic_terms(topicid=0)

[(7, 0.124267705),
 (0, 0.12216605),
 (3, 0.12027098),
 (6, 0.11450522),
 (5, 0.08195062),
 (2, 0.081943735),
 (4, 0.078475155),
 (1, 0.077084005),
 (9, 0.07631046),
 (10, 0.050397925)]

In [11]:
topic2_terms = model.get_topic_terms(topicid=0)
topic2_words = [
    (dictionary.get(i), j)
    for i,j in topic2_terms
]
topic2_words

[('user', 0.124267705),
 ('computer', 0.12216605),
 ('response', 0.12027098),
 ('time', 0.11450522),
 ('system', 0.08195062),
 ('interface', 0.081943735),
 ('survey', 0.078475155),
 ('human', 0.077084005),
 ('trees', 0.07631046),
 ('graph', 0.050397925)]

In [12]:
[
    (dictionary.get(i), j)
    for i,j in model.get_topic_terms(topicid=0)
]

[('user', 0.124267705),
 ('computer', 0.12216605),
 ('response', 0.12027098),
 ('time', 0.11450522),
 ('system', 0.08195062),
 ('interface', 0.081943735),
 ('survey', 0.078475155),
 ('human', 0.077084005),
 ('trees', 0.07631046),
 ('graph', 0.050397925)]

# Zad 

Proszę posortować zdania najbardziej pasujące do danego tematu. 

In [13]:
import gensim

# for d in corpus_lda:
#     print(d)
    
# for d in corpus_lda:
#     print(d[0])

#print(corpus_lda[0][0])

numpy_corpus = gensim.matutils.corpus2dense(corpus_lda, num_terms=2)
docs = np.argsort(numpy_corpus[0,:])[::-1]
for x in docs[:5]:
    print(corpus[x])
    
#trzeba by wypisać raczej zdania niż ich reprezentacje bag-of-words, ale tu nie mam dostepu do tekstu

[(0, 1.0), (3, 1.0), (4, 1.0), (5, 1.0), (6, 1.0), (7, 1.0)]
[(3, 1.0), (6, 1.0), (7, 1.0)]
[(0, 1.0), (1, 1.0), (2, 1.0)]
[(9, 1.0)]
[(2, 1.0), (5, 1.0), (7, 1.0), (8, 1.0)]


# Zad. 
Sprawdzić do jakiego tematu pasuje nowy dokument i jakie są mu najbliższe

In [14]:
doc = "Human computer interaction"

In [15]:
doc_rep = dictionary.doc2bow(doc.split(' '))
# print(doc_rep)
doc_assignments = model[doc_rep]
print(doc_assignments)

[(0, 0.7407068), (1, 0.25929314)]


In [16]:
index = similarities.MatrixSimilarity(corpus_lda)

sims = index[doc_assignments]
print(list(enumerate(sims)))

sims = sorted(enumerate(sims), key=lambda item: -item[1])
print(sims)

[(0, 0.9895207), (1, 0.9744165), (2, 0.54026634), (3, 0.4591714), (4, 0.98414946), (5, 0.69093746), (6, 0.53994584), (7, 0.47826633), (8, 0.49494594)]
[(0, 0.9895207), (4, 0.98414946), (1, 0.9744165), (5, 0.69093746), (2, 0.54026634), (6, 0.53994584), (8, 0.49494594), (7, 0.47826633), (3, 0.4591714)]


# Wizualizacja modelu LDA:

pyLDAvis

http://nbviewer.jupyter.org/github/bmabey/pyLDAvis/blob/master/notebooks/pyLDAvis_overview.ipynb

http://www.kennyshirley.com/LDAvis/

In [17]:
import pyLDAvis.gensim

In [18]:
pyLDAvis.enable_notebook()

In [19]:
# pyLDAvis.gensim.prepare??

In [20]:
pyLDAvis.gensim.prepare(model, corpus, dictionary)

C:\Users\admin\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
1      0.039531  0.0       1        1  53.969139
0     -0.039531  0.0       2        1  46.030861, topic_info=     Category      Freq       Term     Total  loglift  logprob
term                                                          
0     Default  2.000000   computer  2.000000  12.0000  12.0000
3     Default  2.000000   response  2.000000  11.0000  11.0000
6     Default  2.000000       time  2.000000  10.0000  10.0000
8     Default  2.000000        eps  2.000000   9.0000   9.0000
10    Default  2.000000      graph  2.000000   8.0000   8.0000
7     Default  2.000000       user  2.000000   7.0000   7.0000
11    Default  2.000000     minors  2.000000   6.0000   6.0000
5     Default  3.000000     system  3.000000   5.0000   5.0000
2     Default  2.000000  interface  2.000000   4.0000   4.0000
9     Default  2.000000      trees  2.000000   3.0000   3.0000
4     Default  2.000000     survey  2.000000   2.0000   2.0000
1     Default  2.000000      human  2.000000   1.0000   1.0000
8      Topic1  1.685324        eps  2.134009   0.3807  -2.2286
10     Topic1  2.170883      graph  2.843642   0.3468  -1.9754
11     Topic1  1.611536     minors  2.132360   0.3367  -2.2734
5      Topic1  2.454820     system  3.548775   0.2482  -1.8525
9      Topic1  1.817079      trees  2.835743   0.1717  -2.1533
1      Topic1  1.091769      human  2.120759  -0.0472  -2.6627
4      Topic1  1.072775     survey  2.120335  -0.0646  -2.6803
2      Topic1  1.025414  interface  2.119276  -0.1092  -2.7254
7      Topic1  1.162285       user  2.821128  -0.2700  -2.6001
6      Topic1  0.580829       time  2.109354  -0.6729  -3.2938
3      Topic1  0.502106   response  2.107598  -0.8177  -3.4395
0      Topic1  0.476232   computer  2.107020  -0.8704  -3.4924
0      Topic2  1.630788   computer  2.107020   0.5196  -2.1024
3      Topic2  1.605491   response  2.107598   0.5037  -2.1180
6      Topic2  1.528524       time  2.109354   0.4538  -2.1671
7      Topic2  1.658843       user  2.821128   0.2448  -2.0853
2      Topic2  1.093863  interface  2.119276   0.1145  -2.5017
4      Topic2  1.047561     survey  2.120335   0.0707  -2.5450
1      Topic2  1.028990      human  2.120759   0.0527  -2.5629
9      Topic2  1.018664      trees  2.835743  -0.2480  -2.5729
5      Topic2  1.093955     system  3.548775  -0.4009  -2.5016
11     Topic2  0.520825     minors  2.132360  -0.6337  -3.2438
10     Topic2  0.672759      graph  2.843642  -0.6656  -2.9878
8      Topic2  0.448685        eps  2.134009  -0.7836  -3.3929, token_table=      Topic      Freq       Term
term                            
0         2  0.949208   computer
8         1  0.937203        eps
10        1  0.703323      graph
10        2  0.351662      graph
1         1  0.471529      human
1         2  0.471529      human
2         1  0.471859  interface
2         2  0.471859  interface
11        1  0.937928     minors
11        2  0.468964     minors
3         1  0.474474   response
3         2  0.948948   response
4         1  0.471624     survey
4         2  0.471624     survey
5         1  0.563575     system
5         2  0.281787     system
6         1  0.474079       time
6         2  0.948158       time
9         1  0.705282      trees
9         2  0.352641      trees
7         1  0.354468       user
7         2  0.708936       user, R=12, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1])